In [102]:
#1.加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [103]:
#2.定义超参数
BATCH_SIZE = 16 # 128/64，这里是每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 10 #训练数据集的轮次

In [104]:
#3.构建pipeline，对图像作处理
pipeline = transforms.Compose([
    transforms.ToTensor(), #将图片转换成tensor
    transforms.Normalize((0.1307,),(0.3081,)) #正则化，模型出现过拟合现象是，降低模型复杂度
])

In [105]:
#4.下载\加载数据
from torch.utils.data import DataLoader #处理与数据相关的东西

#下载数据集
train_set = datasets.MNIST("data",train = True, download = True, transform = pipeline)
#文件夹名为data,训练集True,下载,处理

test_set = datasets.MNIST("data",train = False, download = True, transform = pipeline)

#加载数据
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
#将train_set的数据集加载，Dataloader加载，batch_size超参数，shuffle打乱
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=True)

In [106]:
#(4)插入代码，显示MNIST中的图片
#查看data文件夹里的图片b
with open("/Users/kongxinyi/Desktop/data/MNIST/raw/train-images-idx3-ubyte","rb")as f:
    file = f.read()

In [107]:
#(4)
image1 = [int(str(item).encode('ascii'),16) for item in file[16 : 16+784]]
# 可以加上print(image1)

In [108]:
#(4)
import cv2
import numpy as np

image1_np = np.array(image1,dtype = np.uint8).reshape(28,28,1)

print(image1_np.shape)

(28, 28, 1)


/var/folders/dg/4_v_lpy53yqb9_mscvxcqrym0000gn/T/ipykernel_16607/3448630253.py:5: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 294 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  image1_np = np.array(image1,dtype = np.uint8).reshape(28,28,1)
/var/folders/dg/4_v_lpy53yqb9_mscvxcqrym0000gn/T/ipykernel_16607/3448630253.py:5: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 310 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  image1_np = np.array(image1,dtype = np.uint8).reshape(28,28,1)
/var/folders/dg/4_v_lpy53yqb9_mscvxcqrym0000gn/T/ipykernel_16607/3448630253.py:5: DeprecationWarning: NumPy will stop allowing conversion of out-o

In [109]:
#(4)
cv2.imwrite("digit.jpg",image1_np)

True

In [110]:
#5.构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5) # 1:灰度图片的通道是1，10:输出通道，5:卷积kernel
        self.conv2 = nn.Conv2d(10,20,3) # 10:输入通道，20:输出通道，3:kernel
        self.fc1 = nn.Linear(20*10*10,500) # 20*10*10：输入通道，500:输出通道
        self.fc2 = nn.Linear(500,10) # 500：输入通道，10：输出通道，0-9的十个类别，分别给出概率
        
    def forward(self,x):
            input_size = x.size(0) # 取size: batch_size输入多少图片 * 1灰度 * 28 * 28像素
            # 0取的是batch_size
            x = self.conv1(x) # 输入：batch*1*28*28,输出：batch*10*24*24
            #原来是28，现在24:28-5+1 5是卷积
            x = F.relu(x) # 激活函数：在所有隐藏层之间添加一个激活函数，输出就是一个非线形函数
            # 也就是提高结果的多样性
            x = F.max_pool2d(x,2,2) # 输入：batch*1*24*24,输出：batch*10*12*12
            
            x = self.conv2(x) # 输入：batch*1*12*12,输出：batch*20*10*10 (12-3+1=10)
            x = F.relu(x)
            
            x = x.view(input_size,-1) # 拉平，-1 自动计算维度，20*10*10
            
            x = self.fc1(x) # 输入：batch*2000 输出：batch*500
            x = F.relu(x) # 保持shape不变
            
            x = self.fc2(x) # 输入：batch*500 输出：batch*10
            
            output = F.log_softmax(x,dim=1)#dim:按行计算
            # 计算分类后，每个数字的概率值
            
            return output
        

In [111]:
#6.定义优化器
model = Digit().to(DEVICE) #创建模型，补数到device设备上

optimizer = optim.Adam(model.parameters()) # 创建优化器，调用adam，更新训练参数，以达到最优值

In [112]:
#7.创建训练方法
def train_model(model,device,train_loader,optimizer,epoch):
    #模型，设备，数据，优化器，epoch跑几次
    #模型训练
    model.train()
    for batch_index,(data,target) in enumerate(train_loader):
        #部署到DEVICE上去:
        data,target = data.to(device),target.to(device)
        #梯度初始化为0:
        optimizer.zero_grad()
        #训练后结果:
        output = model(data)
        #计算损失（差距） 结果与真实值target的差距:
        loss = F.cross_entropy(output,target) #交叉商函数
        # 找到概率值最大的下标:
        pred = output.max(1,keepdim=True)#1:维度/横轴
        #也可以写成 pred = output.argmax(dim=1) argmax:找到最大值下标，dim是纬度
        #进行反向传播：
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch:{}\t Loss:{:.6f}".format(epoch,loss.item()))
            #每3000次打印一次损失

In [113]:
#8.定义测试方法
def test_model(model,device,test_loader):
    #模型验证：
    model.eval()
    #正确率：
    correct = 0.0
    #测试损失：
    test_loss = 0.0
    with torch.no_grad():#不会计算梯度，也不会进行反向传播
        for data,target in test_loader: # 读取数据
            #部署到device上:
            data,target = data.to(device),target.to(device)
            #测试数据:
            output = model(data)
            #计算测试损失:
            test_loss += F.cross_entropy(output,target).item()
            #找到概率最大下标:
            pred = output.max(1,keepdim=True)[1]
            #pred = torch.max(output,dim=1)
            #pred = output.argmax(dim=1)
            #累计正确率:
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test -- Average loss:{:.4f},Accuracy :{:.3f}\n".format(
             test_loss,100.0*correct/len(test_loader.dataset)))

In [114]:
#9.调用方法，输出 调用7/8
for epoch in range(1,EPOCHS + 1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.305202
Train Epoch:1	 Loss:0.067067
Test -- Average loss:0.0032,Accuracy :98.180

Train Epoch:2	 Loss:0.011282
Train Epoch:2	 Loss:0.000112
Test -- Average loss:0.0025,Accuracy :98.800

Train Epoch:3	 Loss:0.031875
Train Epoch:3	 Loss:0.055311
Test -- Average loss:0.0025,Accuracy :98.770

Train Epoch:4	 Loss:0.001135
Train Epoch:4	 Loss:0.006348
Test -- Average loss:0.0030,Accuracy :98.540

Train Epoch:5	 Loss:0.000042
Train Epoch:5	 Loss:0.000007
Test -- Average loss:0.0029,Accuracy :98.920

Train Epoch:6	 Loss:0.000089
Train Epoch:6	 Loss:0.000012
Test -- Average loss:0.0025,Accuracy :99.030

Train Epoch:7	 Loss:0.001933
Train Epoch:7	 Loss:0.000161
Test -- Average loss:0.0027,Accuracy :99.120

Train Epoch:8	 Loss:0.000000
Train Epoch:8	 Loss:0.000093
Test -- Average loss:0.0031,Accuracy :99.030

Train Epoch:9	 Loss:0.000184
Train Epoch:9	 Loss:0.002907
Test -- Average loss:0.0041,Accuracy :99.050

Train Epoch:10	 Loss:0.000000
Train Epoch:10	 Loss:0.000000
Test